In [ ]:
# airflow를 이용해서 테스트 진행

#!airflow initdb
!airflow test DAG_model_training emr_cluster_create 2019-08-01
#!airflow test DAG_model_training emr_ssh_connect 2019-08-01
#!airflow test DAG_model_training ip_forwarding 2019-08-01
#!airflow test DAG_model_training traindata_s3_to_hdfs 2019-08-01
#!airflow test DAG_model_training training_model 2019-08-01
#!airflow test DAG_model_training trained_model_hdfs_to_s3 2019-08-01
#!airflow test DAG_model_training trained_model_s3_to_server 2019-08-01
#!airflow test DAG_model_training emr_cluster_close 2019-08-01

In [1]:
# 환경변수 변경(쥬피터 실행하는 콘솔에 복붙) 
# !! 쥬피터 내에서 실행하는 것은 새로고침하면 오류뜬다.
export HADOOP_CONF_DIR='/home/ubuntu/project1/hadoop/etc/hadoop-model'
export YARN_CONF_DIR='/home/ubuntu/project1/hadoop/etc/hadoop-model'

!echo $HADOOP_CONF_DIR

/home/ubuntu/project1/hadoop/etc/hadoop-model


In [1]:
import sys, os, re, json, datetime, iso8601, timeit
from tabulate import tabulate

import pyspark
import pyspark.sql

# EMR 클러스터 사용할 때 설정
#conf = pyspark.SparkConf().setAll([('spark.master', 'yarn'),("spark.driver.memory", "2g"),('spark.executor.memory', '18g'), ('spark.executor.cores', '4'), ('spark.executor.instances', '4'),('spark.submit.deployMode','client')])

# EMR 클러스터 사용 안 할때 설정
conf = pyspark.SparkConf().setAll([('spark.driver.memory', '2g')])

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc).builder.getOrCreate()
    
# 로컬로 실행한다면    
PROJECT_PATH = '../../02_Data_Batch_Processing'

# emr에서 실행한다면
#PROJECT_PATH = ''

In [2]:
# pyspark 설정 보기
pyspark.SparkConf().getAll()

[('spark.executor.instances', '4'),
 ('spark.repl.local.jars',
  'file:///home/ubuntu/project1/lib/mongo-hadoop-spark-2.0.2.jar,file:///home/ubuntu/.ivy2/jars/org.apache.spark_spark-streaming-kafka-0-8_2.11-2.4.0.jar,file:///home/ubuntu/.ivy2/jars/org.apache.spark_spark-streaming-kinesis-asl_2.11-2.4.0.jar,file:///home/ubuntu/.ivy2/jars/org.apache.kafka_kafka_2.11-0.8.2.1.jar,file:///home/ubuntu/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar,file:///home/ubuntu/.ivy2/jars/org.scala-lang.modules_scala-xml_2.11-1.0.2.jar,file:///home/ubuntu/.ivy2/jars/com.yammer.metrics_metrics-core-2.2.0.jar,file:///home/ubuntu/.ivy2/jars/org.scala-lang.modules_scala-parser-combinators_2.11-1.1.0.jar,file:///home/ubuntu/.ivy2/jars/com.101tec_zkclient-0.3.jar,file:///home/ubuntu/.ivy2/jars/org.apache.kafka_kafka-clients-0.8.2.1.jar,file:///home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar,file:///home/ubuntu/.ivy2/jars/log4j_log4j-1.2.17.jar,file:///home/ubuntu/.ivy2/jars/net.jpountz.lz4_lz4-1.2

# 데이터 가져오기

In [2]:
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, DateType, TimestampType
from pyspark.sql.types import StructType, StructField
from pyspark.sql.functions import udf

SEED = 2019

# 원본 데이터(널값 제거 안함)
input_path = "{}/data/Raw_Data.parquet".format(PROJECT_PATH)

# 컬럼 필터한 데이터(널값 제거) 
#input_path = "{}/data/Refined_Data.parquet".format(PROJECT_PATH)

data_set = spark.read.parquet(input_path)

# 데이터 크기를 줄이기 위한 샘플 채취 
data_set = data_set.sample(False, 0.1)

data_set.first()

Row(Year='2015', Quarter='2', Month='5', DayofMonth='9', DayOfWeek='6', FlightDate='2015-05-09', Carrier='WN', TailNum='N8311Q', FlightNum='1839', Origin='MCO', OriginCityName='Orlando, FL', OriginState='FL', Dest='SJU', DestCityName='San Juan, PR', DestState='PR', DepTime='1118', DepDelay=-2.0, DepDelayMinutes=0, TaxiOut=8.0, TaxiIn=9.0, WheelsOff='1126', WheelsOn='1352', ArrTime='1401', ArrDelay=-4.0, ArrDelayMinutes=0.0, Cancelled=0, Diverted=0, ActualElapsedTime=163.0, AirTime=146.0, Flights=1, Distance=1189.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None, CRSDepTime='1120', CRSArrTime='1405')

In [3]:
# 데이터 개수 확인
data_set.count()

582692

# null 값 제거

In [4]:
# 널값 검사
null_counts = [(column, data_set.where(data_set[column].isNull()).count()) for column in data_set.columns]
cols_with_nulls = list(filter(lambda x: x[1] > 0, null_counts))
print(cols_with_nulls)

# 넓값 지우기 전
print("지우기 전:",data_set.count())

# 만약 널값이 있다면 제거 한다.
if list(cols_with_nulls):    
    data_set = data_set.na.drop()

print("지운 후:",data_set.count())

[('TailNum', 1455), ('DepTime', 8684), ('DepDelay', 8684), ('DepDelayMinutes', 8684), ('TaxiOut', 8954), ('TaxiIn', 9294), ('WheelsOff', 8954), ('WheelsOn', 9294), ('ArrTime', 9294), ('ArrDelay', 10602), ('ArrDelayMinutes', 10602), ('ActualElapsedTime', 10602), ('AirTime', 10602), ('CarrierDelay', 476032), ('WeatherDelay', 476032), ('NASDelay', 476032), ('SecurityDelay', 476032), ('LateAircraftDelay', 476032)]
지우기 전: 582692
지운 후: 106660


# 훈련 데이터 만들기

In [6]:
# 테이블 등록
data_set.registerTempTable("data_set")

# 모델 훈련에 쓰일 데이터 생성
training_yet_data = spark.sql("""
SELECT
  FlightNum,
  FlightDate,
  DayOfWeek,
  DayofMonth AS DayOfMonth,
  CONCAT(Month, '-',  DayofMonth) AS DayOfYear,
  Carrier,
  Origin,
  Dest,
  Distance,
  DepDelay,
  ArrDelay,
FROM data_set
""")

sys.path.append("../../02_Data_Batch_Processing/lib")
import date_util

# SparkContext에 모듈 등록
sc.addPyFile('../../02_Data_Batch_Processing/lib/date_util.py')

# date_util.alter_feature_datetimes : 날짜 파싱
training_yet_data=training_yet_data.rdd.map(date_util.alter_feature_datetimes).toDF()
training_yet_data.show(2)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+-------------------+---------+------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|         FlightDate|FlightNum|Origin|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+-------------------+---------+------+
|    72.0|2015-05-09 13:45:00|2015-05-09 11:55:00|     WN|         9|        6|      129|    79.0| MDW|  1105.0|2015-05-09 00:00:00|      973|   RSW|
|    45.0|2015-05-09 19:35:00|2015-05-09 17:40:00|     WN|         9|        6|      129|     9.0| DEN|  1506.0|2015-05-09 00:00:00|      962|   TPA|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+-------------------+---------+------+
only showing top 2 rows



# 훈련 데이터 전처리

In [ ]:
# 항공편 번호를 운항 경로로 대체하기
from pyspark.sql.functions import lit, concat

features_with_route = training_yet_data.withColumn(
  'Route',
  concat(
    training_yet_data.Origin,
    lit('-'),
    training_yet_data.Dest
  )
)

#### Bucketizer:목표변수 분류 클래스 나누기 ####
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), -15.0, 0, 30.0, float("inf")]
bucketizer = Bucketizer(
  splits=splits,
  inputCol="ArrDelay", #원시 목표변수
  outputCol="ArrDelayBucket" #클래스 나뉜 목표변수
)

# Bucketizer로 데이터 변환
ml_bucketized_features = bucketizer.transform(features_with_route)

#### StringIndexer : String 타입의 범주 값을 해당 값의 정수 번호로 변환 ####
from pyspark.ml.feature import StringIndexer

for column in ["Carrier", "Origin", "Dest", "Route"]:
    string_indexer = StringIndexer(
    inputCol=column,
    outputCol=column + "_index"
    )
    string_indexer_model = string_indexer.fit(ml_bucketized_features)
    ml_bucketized_features = string_indexer_model.transform(ml_bucketized_features)

    ml_bucketized_features = ml_bucketized_features.drop(column)


#### VectorAssembler: 데이터를 벡터화 하기 ####
from pyspark.ml.feature import VectorAssembler

numeric_columns = ["DepDelay", "Distance",
    "DayOfMonth", "DayOfWeek",
    "DayOfYear"]
index_columns = ["Carrier_index", "Origin_index",
                   "Dest_index", "Route_index"]
vector_assembler = VectorAssembler(
  inputCols=numeric_columns + index_columns,
  outputCol="Features_vec"
)
final_vectorized_features = vector_assembler.transform(ml_bucketized_features)

# 필요없는 컬럼 제거
for column in index_columns:
    final_vectorized_features = final_vectorized_features.drop(column)

# 모델 훈련하기

하이퍼 파라미터 수정 x = 약 59%


수정(깊이 10, 개수 5) = 약 61% / 걸린시간 약 7분

EMR(m1,d2)+수정(깊이 20, 개수 10) = 애러 / 걸린시간 2.7시간 / 이유 : 메모리 부족

EMR(m1,d2)+수정(깊이 10, 개수 5) = 약 61% / 걸린시간 약 4분 

EMR(m1,d2)+수정(깊이 15, 개수 8) = 약 63% / 걸린시간 약 26분

EMR(m1,d2)+수정(깊이 15, 개수 20) = 약 63% / 걸린시간 약 63분

EMR(m1,d2)+수정(깊이 15, 개수 20)+원본의10% = 약 62% / 걸린시간 약 22분

EMR(m1,d2,j2)+수정(깊이 15, 개수 20)+원본의10% = 약 62% / 걸린시간 약 17분

EMR(m1,d2,j2)+수정(깊이 18, 개수 20)+원본의10% = 약 59% / 걸린시간 약 39분 !깊이 올렸지만 오히려 떨어짐

EMR(m1,d2,j2)+수정(깊이 20, 개수 5)+원본의10% + 전체실행자수 4->5 = 약 59% / 걸린시간 약 16분

EMR(m1,d2,j2)+수정(깊이 20, 개수 10)+원본의10% + 전체실행자수 4->5 = 약 61% / 걸린시간 약 31분

EMR(m1,d2,j2)+수정(깊이 30, 개수 5)+원본의10% + 전체실행자수 4->5 = 약 57% / 걸린시간 약 40분

EMR(m1,d2,j2)+수정(깊이 10, 개수 5)+원본의10% + 전체실행자수 4->5 = 약 60% / 걸린시간 약 1분

EMR(m1,d2,j2)+수정(깊이 10, 개수 5)+원본의100% + 전체실행자수 4->5 = 약 60% / 걸린시간 약 1분

EMR(m1,d2,j2)+수정(깊이 10, 개수 5)+원본의100% + 전체실행자수 4->5 = 약 61% / 걸린시간 약 4분

EMR(m1,d2,j2)+수정(깊이 10, 개수 5)+원본의100% + 전체실행자수 4->5 = 약 61% / 걸린시간 약 3분

1. 클러스터 추가 할 수록 속도는 빨라짐.(다만 처리해야하는 양이 많이 않을 때는 비슷 함. 각 노드를 확인해 보니 양이 적을 때는 놀고 있는 노드가 있었음(cpu %를 보고 판단)

2. 깊이를 늘리면 실행시간은 '제곱'으로 느려지는 것 같음 ( 깊이 15 -> 18로 늘렸을 때, 실행시간은 약 2배 늘어남 )

3. 개수를 늘리면 실행시간은 '배수'로  느려지는 것 같음 (예 개수 5 -> 10으로 2배 늘렸을 때 실행시간도 약 2배 늘어남)

4. 일정 수준 데이터 수가 충족되면(아니면 비슷한 데이터가 많아서 거품낀 데이터 일 수도) 모델 성능에 크게 영향을 안줌 (예 원본의 100%와 10%로 훈련했을 때 정확도는 크게 차이없음.)

5. 깊이는 높아질 수록 '과적합'되는 것같음 (깊이 10, 개수 5일때(정확도 61와 깊이 30, 개수 5 일때 비교 했을 때 '전자'가 정확도가 높았음)

6. 개수는 일정수준에 도달하면 더 이상 성능이 증가하지 않음(8개에서 20개로 늘렸는데 정확도는 거이 차이가 없었음.)

7. 실행자수 4에서 5로 늘리고 노드도 추가했지만  걸린시간은 비슷했음.. ( 1번과 같은 이유 인듯)



In [7]:
# 모델 훈련(8:2)
training_data, test_data = final_vectorized_features.randomSplit([0.8, 0.2],seed=SEED)
training_data.cache()
test_data.cache()

DataFrame[ArrDelay: double, CRSArrTime: timestamp, CRSDepTime: timestamp, DayOfMonth: bigint, DayOfWeek: bigint, DayOfYear: bigint, DepDelay: double, Distance: double, FlightDate: timestamp, FlightNum: string, ArrDelayBucket: double, Features_vec: vector]

In [16]:
# 모델 : 랜덤포레스트
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(
  featuresCol="Features_vec",
  labelCol="ArrDelayBucket",
  numTrees=10,
  maxBins=4657,
  maxDepth=15,
  maxMemoryInMB=1048,
  seed = SEED,
  #checkpointInterval = 10,
  cacheNodeIds = True,
  featureSubsetStrategy="auto"
    
)
# cache_node_ids = TRUE
# checkpoint_interval = 10
# featureSubsetStrategy="auto"

In [17]:
# 훈련시작(시간 체크)
start = timeit.default_timer()

model = rfc.fit(training_data)

stop = timeit.default_timer()
print((stop - start)/60) 

2.5918552185500023


In [18]:
# Evaluate model using test data
predictions = model.transform(test_data)
predictions.show(2)

+--------+-------------------+-------------------+----------+---------+---------+--------+--------+-------------------+---------+--------------+--------------------+--------------------+--------------------+----------+
|ArrDelay|         CRSArrTime|         CRSDepTime|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Distance|         FlightDate|FlightNum|ArrDelayBucket|        Features_vec|       rawPrediction|         probability|prediction|
+--------+-------------------+-------------------+----------+---------+---------+--------+--------+-------------------+---------+--------------+--------------------+--------------------+--------------------+----------+
|   -82.0|2015-01-21 20:49:00|2015-01-21 17:20:00|        21|        3|       21|   -15.0|  2402.0|2015-01-21 00:00:00|       11|           0.0|[-15.0,2402.0,21....|[2.65697552471158...|[0.53139510494231...|       0.0|
|   -80.0|2015-11-17 19:57:00|2015-11-17 17:00:00|        17|        2|      321|   -13.0|  2248.0|2015-11-17 00:00:00|     

In [19]:
# 훈련된 모델에 테스트 데이터로 검증하기
predictions = model.transform(test_data)

# 정확도 점수
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6105440467272318


In [12]:
# f1점수
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="ArrDelayBucket", metricName="f1")
accuracy = evaluator.evaluate(predictions)
print("f1 = {}".format(accuracy))

f1 = 0.6100615729464689


# 교차 검증

데이터 전처리하고 난 뒤

In [8]:
# 모델 훈련(8:2)
from numpy import *
random.seed(SEED)

training_data, test_data = final_vectorized_features.randomSplit([0.8, 0.2])
training_data.cache()
test_data.cache()

DataFrame[ArrDelay: double, CRSArrTime: timestamp, CRSDepTime: timestamp, DayOfMonth: bigint, DayOfWeek: bigint, DayOfYear: bigint, DepDelay: double, Distance: double, FlightDate: timestamp, FlightNum: string, ArrDelayBucket: double, Features_vec: vector]

In [17]:
# k겹 교차 검즘
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 반복 횟수
numFolds = 5 

# 모델 정의(랜덤 포레스트)
rf = RandomForestClassifier(    
    featuresCol="Features_vec",
    labelCol="ArrDelayBucket",
    predictionCol="Prediction",
    maxBins=4657,
    maxMemoryInMB=1024)

# 모델 평가 정의
evaluator = MulticlassClassificationEvaluator(
            labelCol="ArrDelayBucket",
            predictionCol="Prediction") 

# 파이프 라인 및 모델 파라미터 정의
pipeline = Pipeline(stages=[rf]) # stages 배열 순서대로 나중에 bestModel에서 속성들을 찾을 수 있다.
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [10,15,20]).addGrid(rf.maxDepth, [5,7,10]).build()

# 교차 검증 정의
crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=numFolds)

# 훈련시작(시간 체크)
start = timeit.default_timer()

model = crossval.fit(trainingData)

stop = timeit.default_timer()
print('걸림시간: ',(stop - start)/60) 

11.455859561233334


# 베스트 모델 다루기

In [64]:
# 모델 중 가장 좋은 성능의 모델 가져오기
bestPipeline = model.bestModel
print(bestPipeline)

# stages 반환값은 배열. -1 이라는 것은 가장 마지막 열 len-1 과 같다. # 그리고 stages는 위에서 만든 pipline의 매개변수이다.
# 파이프라인의 랜덤 포레스트 모델 가져오기
bestModel = bestPipeline.stages[0]
print(bestModel)

# 특징 중요도 
importances = bestModel.featureImportances
print(importances)

# 모델의 트리개수(아래 두 코드는 같은 결과)
print(bestModel.getNumTrees)
print(model.bestModel.stages[-1]._java_obj.getNumTrees())

# 모델의 트리 깊이(아래 두 코드는 같은 결과)
print(bestModel.getOrDefault('maxDepth'))
print(model.bestModel.stages[len(model.bestModel.stages)-1]._java_obj.getMaxDepth())

PipelineModel_5933924c7a53
RandomForestClassificationModel (uid=RandomForestClassifier_ef9d13b13603) with 10 trees
(9,[0,1,2,3,4,5,6,7,8],[0.856746453954314,0.02405667307481816,0.0018408220086847758,0.0011827426965437596,0.007994231896378986,0.014629792815877821,0.019966781962298414,0.01446072462429139,0.059121776966792584])
10
10
10
10


In [135]:
# 배스트 모델의 정확도 확인
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = {"Accuracy":evaluator.evaluate(prediction)}

print("Accuracy = {}".format(accuracy["Accuracy"]))



Accuracy = 0.6104931941486331


# 특징 중요도

In [165]:
# 빈 리스트 생성
feature_importances = defaultdict(list)

# 컬럼명 가져오기
feature_names = vector_assembler.getInputCols()
# 특징 중요도
feature_importance_list = bestModel.featureImportances

# zip 함수로 컬럼명과 특징 중요도록 묶어준다.
for feature_name, feature_importance in zip(feature_names, feature_importance_list):
    feature_importances[feature_name].append(feature_importance)

# 리스트로 묶여있는 특징 중요도를 하나의 값,그리고 float 형식으로 변환
feature_importance_entry = defaultdict(float)
for feature_name, value_list in feature_importances.items():
  average_importance = sum(value_list) / len(value_list)
  feature_importance_entry[feature_name] = average_importance

# 정렬하기
import operator
sorted_feature_importances = sorted(
  feature_importance_entry.items(), # 정렬할 데이터
  key=operator.itemgetter(1), # 정렬할 키(0:컬럼이름,1:특징 중요도)
  reverse=True # 내림차순
)

print("\n정렬된 특징 중요도")
print("-------------------")
print(tabulate(sorted_feature_importances, headers=['Name', 'Importance'])) # tabulate: 시각적을 보기 좋게 테이블화 해주는 모듈



정렬된 특징 중요도
-------------------
Name             Importance
-------------  ------------
DepDelay         0.858286
Route_index      0.0668056
Origin_index     0.0217321
Distance         0.0194487
Dest_index       0.0123101
Carrier_index    0.0113784
DayOfYear        0.00731536
DayOfMonth       0.00160242
DayOfWeek        0.00112085


# 이전 결과와 비교(정확도,특징 중요도)

In [113]:
# 현제 정확도, 특징 중요도 점수 가져와서 합치기
score_log_entry = {result_name: accuracy[result_name] for result_name in result_names}
for feature_name, importance in sorted_feature_importances:
    score_log_entry[feature_name] = importance

score_log_entry

{'Accuracy': 0.6104931941486331,
 'DepDelay': 0.858286342821538,
 'Route_index': 0.06680557136144623,
 'Origin_index': 0.021732143046945946,
 'Distance': 0.01944874148171331,
 'Dest_index': 0.012310135720946623,
 'Carrier_index': 0.011378438035375332,
 'DayOfYear': 0.007315360251440876,
 'DayOfMonth': 0.0016024150398984331,
 'DayOfWeek': 0.0011208522406953548}

In [155]:
import pickle, copy

# 정확도, 특징 중요도를 합친 리스트 만들기
result_names = list(accuracy.keys()) # 정확도 말고 f1 같은것도 넣을 수 있다.
feature_names = vector_assembler.getInputCols()

metric_names=[]
for result_name in result_names:
    metric_names.append(result_name) 

for feature_name in feature_names:
    metric_names.append(feature_name)

# 이전 결과값 가져오기
try:
  score_log_filename = "./models/score_log.pickle"
  score_log = pickle.load(open(score_log_filename, "rb"))
  if not isinstance(score_log, list):
    score_log = []
except IOError:
  score_log = []

# 현제 정확도, 특징 중요도 점수 가져와서 합치기
score_log_entry = copy.deepcopy(accuracy)
for feature_name, importance in sorted_feature_importances:
    score_log_entry[feature_name] = importance

# 이전 결과값이 있다면 이전 결과값 반환, 이전 결과값이 없다면 현재 결과값 반환.
try:
  last_log = score_log[-1]
  print("이전기록 있음")
except (IndexError, TypeError, AttributeError):
  last_log = score_log_entry
  print("이전기록 없음")

experiment_report = []
for metric_name in metric_names:
  run_delta = score_log_entry[metric_name] - last_log[metric_name]
  experiment_report.append((metric_name, run_delta))

print("\n이전 기록과 차이")
print("-----------------")
print(tabulate(experiment_report, headers=["Metric", "Score"]))

# 이전 결과값에 현재 결과값을 붙친다.
score_log.append(score_log_entry)

# 객체로 저장.
pickle.dump(score_log, open(score_log_filename, "wb"))

이전기록 있음

Experiment Report
-----------------
Metric           Score
-------------  -------
Accuracy             0
DepDelay             0
Distance             0
DayOfMonth           0
DayOfWeek            0
DayOfYear            0
Carrier_index        0
Origin_index         0
Dest_index           0
Route_index          0


In [146]:
# 이전 데이터는 덮어쓰지 않고 추가한다.
score_log

[{'Accuracy': 0.6104931941486331,
  'DepDelay': 0.858286342821538,
  'Route_index': 0.06680557136144623,
  'Origin_index': 0.021732143046945946,
  'Distance': 0.01944874148171331,
  'Dest_index': 0.012310135720946623,
  'Carrier_index': 0.011378438035375332,
  'DayOfYear': 0.007315360251440876,
  'DayOfMonth': 0.0016024150398984331,
  'DayOfWeek': 0.0011208522406953548},
 {'Accuracy': 0.6104931941486331,
  'DepDelay': 0.858286342821538,
  'Route_index': 0.06680557136144623,
  'Origin_index': 0.021732143046945946,
  'Distance': 0.01944874148171331,
  'Dest_index': 0.012310135720946623,
  'Carrier_index': 0.011378438035375332,
  'DayOfYear': 0.007315360251440876,
  'DayOfMonth': 0.0016024150398984331,
  'DayOfWeek': 0.0011208522406953548}]

# SEED 적용 범위 

데이터 훈련/테스트로 분할 할 때는 seed 적용 x, 모델 파라미터 seed 적용 o
데이터 훈련/테스트로 분할 할 때는 seed 적용 o, 모델 파라미터 seed 적용 o
=> 두 훈련 결과는 다름 

데이터 훈련/테스트로 분할 할 때는 seed 적용 o, 모델 파라미터 seed 적용 x
데이터 훈련/테스트로 분할 할 때는 seed 적용 o, 모델 파라미터 seed 적용 x
=> 두 훈련 결과는 같음

결론 훈련 데이터의 seed가 다르면 결과도 다르게 나오는데, 모델 파라미터 seed는 달라도 결과 같음.

그리고

통합적으로 seed를 걸수 있는데
from numpy import *
random.seed(SEED)

위 코드를 시작하는 부분에 넣으면 그 뒤로 나오는 랜덤함수를 호출하는 부분은 

설정한 시드값을 기준으로 출력된다.

In [11]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터x, 모델 0)
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6124855450818236


In [16]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터0, 모델 0)
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6095072829573539


In [18]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터0, 모델 x) 시도1 약 10분걸림
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6101470614012025


In [20]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터0, 모델 x) 시도2 약 10분
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6101470614012025


In [24]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터x, 모델 x, numpy으로 지정) 시도1 약 10분
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6098067987282955


In [26]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터x, 모델 x, numpy으로 지정) 시도2 약 10분
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6098067987282955


In [29]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터x, 모델 x, numpy으로 지정(2019+1)) 시도1 약 10분
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6081872373349276


In [31]:
prediction = model.bestModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol="Prediction",labelCol="ArrDelayBucket", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
# 깊이 10 개수 10 시드(데이터x, 모델 x, numpy으로 지정(2019+1)) 시도2 약 10분
print("Accuracy = {}".format(accuracy))

Accuracy = 0.6081872373349276


-------------------end---------------------------------------------

In [52]:
# spark 공식 홈피 모델 튜닝 예제
# https://spark.apache.org/docs/2.4.0/ml-tuning.html#model-selection-aka-hyperparameter-tuning
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Prepare training documents, which are labeled.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

# Prepare test documents, which are unlabeled.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    print(row)

Row(id=4, text='spark i j k', probability=DenseVector([0.2581, 0.7419]), prediction=1.0)
Row(id=5, text='l m n', probability=DenseVector([0.9186, 0.0814]), prediction=0.0)
Row(id=6, text='mapreduce spark', probability=DenseVector([0.432, 0.568]), prediction=1.0)
Row(id=7, text='apache hadoop', probability=DenseVector([0.6766, 0.3234]), prediction=0.0)
